In [14]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji
from sklearn.metrics import *

In [2]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i])

In [3]:
def preprocess_text(tweet, flag_stemm=False, flag_lemm=False, stop_words_list=None):
#     print(tweet)
    tweet=emoji.demojize(tweet)
    tweet=tweet.replace(":"," ")
#     print(tweet)

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
#     print(tweet)

    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

In [7]:
df_training=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
df_training.drop(columns="id",inplace=True)
df_training
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])



# df_testing=pd.DataFrame(pd.read_csv("testset-levela.tsv",sep="\t"))
# df_testing['tweet_original']=df_testing['tweet']
# preprocess_text_caller(df_testing['tweet'])


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range parameter (1,2) means that unigram and bigram will be taken
# Count Vecotrizer automatically preprocess the tweets
vectorizer = TfidfVectorizer(ngram_range=(1,4),min_df=5)

In [25]:
Train_ngrams=vectorizer.fit_transform(df_training.tweet)


NameError: name 'f1_score' is not defined

In [16]:
df_validation=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))
df_validation.drop(columns="id",inplace=True)
df_validation
df_validation['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_validation['tweet'])


In [26]:
Validation_ngrams=vectorizer.transform(df_validation.tweet)


In [23]:
Validation_ngrams

<2140x3250 sparse matrix of type '<class 'numpy.float64'>'
	with 59317 stored elements in Compressed Sparse Row format>

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)


In [32]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9096900112149515

In [33]:
accuracy_score(y_pred,df_validation["label"])

0.9098130841121496

In [34]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
clf.fit(Train_ngrams,df_training["label"])
y_pred=clf.predict(Validation_ngrams)
